In [ ]:
import requests

In [ ]:
auth = '1hMEqSwhQWWRo88SupMVzQ==', '6b2ffab0-5916-4a20-a0d7-8f9824627d7b'

In [ ]:
from httplib import HTTPConnection
import json

known_services = [
'BusStops'
]

credentials = {
    "api_token": None,
    "uuid": None
}

def authenticate(api_token, uuid):
    credentials['api_token'] = api_token
    credentials['uuid'] = uuid

def get(service, start = 0, end=None):
    conn = HTTPConnection('datamall2.mytransport.sg')

    return_value = []

    while end == None or start < end:
        print  '/ltaodataservice/%s?$skip=%i' % (service, start)
        conn.request('GET', '/ltaodataservice/%s?$skip=%i' % (service, start),
            headers = {
                'AccountKey': credentials['api_token'],
                'UniqueUserId': credentials['uuid'],
                'Accept': 'application/json'
            })

        resp = conn.getresponse()
        headers = resp.getheaders()
        body = resp.read()

        data = json.loads(body)

        return_value += data['value']

        if len(data['value']) == 0:
            break

        start += 50

    return return_value

In [ ]:
authenticate(*auth)

In [ ]:
BusRoutes = get('BusRoutes')

In [ ]:
BusStops = get('BusStops')

In [ ]:
import itertools as it

stopsByRoute = [ (k, list(v)) for k, v in it.groupby(BusRoutes, lambda x : (x['ServiceNo'], x['Direction']))]
stopsByRoute = [ (k, sorted(v, key=lambda x:x['StopSequence'])) for k,v in stopsByRoute]
stopsByRoute[0]

In [ ]:
# obtain the (prev, next) pairs
def prev_next(bus_stops):
    for a,b in zip(bus_stops, bus_stops[1:]):
        yield a['BusStopCode'], b['BusStopCode']

stop_pairs = [list(prev_next(sbr[1])) for sbr in stopsByRoute]
stop_pairs = [x for y in stop_pairs for x in y ]
stop_pairs = [x for x in stop_pairs if not x[0].startswith('E') and not x[1].startswith('E')]
stop_pairs = list(set(stop_pairs))

In [ ]:
import pyproj
svy = pyproj.Proj(init='epsg:3414')

In [ ]:
# pull additional bus stops from One Map
import gzip
import pickle

om_bus_stops = pickle.load(gzip.open('../1mbs/busstops2.pickle.gz'))
om_bus_stops = [r['SearchResults'][1:] for r in om_bus_stops if len(r['SearchResults']) > 1]
om_bus_stops = [x for y in om_bus_stops for x in y]
om_bus_stops = [x for x in om_bus_stops if x['SEARCHVAL'].endswith(' (BUS STOP)')]
for bs in om_bus_stops:
    x,y = bs['X'], bs['Y']
    lng,lat = svy(x,y, inverse=True)
    
    bs['Latitude'] = lat
    bs['Longitude'] = lng
    bs['BusStopCode'] = bs['SEARCHVAL'][0:5]
    bs['Description'] = 'Bus Stop No. %s' % bs['BusStopCode']
    bs['RoadName'] = '??'
    

om_bus_stops

In [ ]:
combined = om_bus_stops + BusStops

# print [x for x in combined if x['BusStopCode'] == '42885']

combined = [c for c in combined if c['Longitude'] != 0]
stop_lat_lngs = {stop['BusStopCode']: stop for stop in combined}

[s for s in combined if 'blk 3b' in s['Description'].lower()]

In [ ]:
# http://localhost:8989/route?
#point=1.373846%2C103.880539&point=1.305887%2C103.805008&
#type=json&key=&locale=en-US&vehicle=car&weighting=fastest&elevation=false

In [ ]:
import requests
import polyline

In [ ]:
def make_request(pair):
    ll1 = stop_lat_lngs[pair[0]]
    ll2 = stop_lat_lngs[pair[1]]
    
    return ['%s,%s' % (ll1['Latitude'], ll1['Longitude']),
            '%s,%s' % (ll2['Latitude'], ll2['Longitude'])]


def get_route(stop_pair):
    res = requests.get('http://localhost:8989/route',
        params={
            "point": [
                make_request(stop_pair)
            ],
            "type": "json",
            "vehicle": "car",
            "weighting": "fastest",
        })
    
    res = json.loads(res.text)
    
    if not 'paths' in res:
        return None
    
    return polyline.decode(res['paths'][0]['points'])

def has_stop_location(stop_pair):
    return stop_pair[0] in stop_lat_lngs \
        and stop_pair[1] in stop_lat_lngs

pair_routes = [(get_route(stop_pair), stop_pair) for stop_pair in stop_pairs if has_stop_location(stop_pair)]
pair_routes = [x for x in pair_routes if x[0] != None]
[a for a in pair_routes if a[1][0] == '14409']

In [ ]:
[a for a in pair_routes if a[1][1] == '14409'], \
[s for s in combined if s['BusStopCode'] == '15049'], \
[s for s in combined if s['BusStopCode'] == '14419'], \

In [ ]:
import folium
m = folium.Map(location=[1.38, 103.8], zoom_start=12)
m.line(pair_routes[100][0])
m

In [ ]:
import math

def swap((x,y)):
    return (y,x)

def heading(ll1, ll2):
    xy1 = svy(*swap(ll1))
    xy2 = svy(*swap(ll2))
    
    return (xy2[0] - xy1[0], xy2[1] - xy1[1])

def dedup(ls):
    return [x for x,y in it.groupby(ls)]

deduped_pair_routes = [
    (dedup(path),stops) for path,stops in pair_routes
]

beginning_headings = [
    (heading(path[0], path[1]), stops[0]) for path,stops in deduped_pair_routes
]
end_headings = [
    (heading(path[-2], path[-1]), stops[1]) for path,stops in deduped_pair_routes
]

headings = beginning_headings + end_headings

In [ ]:
stop_headings = {x: list([z[0] for z in y]) for x,y in it.groupby(sorted(headings, key=lambda x:x[1]), key=lambda x:x[1])}

In [ ]:
stop_headings['14409']

math.atan2(7.787889799787081, -210.09228901940878) / math.pi * 180

In [ ]:
multiple = [(s,x) for s,x in stop_headings.items() if len(x) > 1]

In [ ]:
def cosine_distance_to_first(ls):
    def cosine_distance((a,b), (c,d)):
        return (a*c + b*d) / math.sqrt(a*a + b*b) / math.sqrt(c*c + d*d)
    
    return [
        cosine_distance(ls[0], l) for l in ls[1:]
    ]

multiple_headings = [ (s, cosine_distance_to_first(l)) for s,l in multiple]

In [ ]:
import numpy as np
l = [ (s, similarities) for s, similarities in multiple_headings if np.min(similarities) < 0.9]
l

In [ ]:
[s[0] for s in l]

In [ ]:
# Perform heading annotations

from pymongo import MongoClient

client = MongoClient()
db = client.routing

results = db.busStopHeading.find({
        "heading": {"$exists": True},
        "BusStopCode": {"$in": [s[0] for s in l]},
        "Longitude": {"$ne": 0}
    })
heading_annotations = list(results)
heading_annotations_by_stop = {a['BusStopCode']: a for a in heading_annotations}
heading_annotations

In [ ]:
import numpy as np

# Get the final list...
def average_heading(xys):
    acc = np.array([0.0, 0.0])
    
    for xy in xys:
        xya = np.array(xy)
        xy_normalized = xy / np.linalg.norm(xya)
        
        acc += xy_normalized
        
    return math.atan2(*acc)

calculated = [ (s, average_heading(x)) for s, x in stop_headings.items()]

def infer_heading_from_annotation(a):
    (lat1,lng1), (lat2,lng2) = a['heading']
    (lat,lng) = a['Latitude'], a['Longitude']

    (xy1, xy2) = svy(lng1, lat1), svy(lng2, lat2)

    return math.atan2(xy2[0] - xy1[0], xy2[1] - xy1[1])
     
     
annotated = [ (a['BusStopCode'], infer_heading_from_annotation(a)) for a in heading_annotations ]

all_headings = dict(calculated + annotated)
all_headings['14409']

In [ ]:
# check the sanity of the calculated stop headings
idx = 0
c = stop_lat_lngs[calculated[idx][0]]
print calculated[idx][1] / math.pi * 180

loc = (c['Latitude'], c['Longitude'])
m = folium.Map(location=loc, zoom_start=18)
m.simple_marker(location=loc)
m
            

In [ ]:
# check the sanity of the annotations
idx = 10
c = stop_lat_lngs[annotated[idx][0]]
print annotated[idx][1] / math.pi * 180

loc = (c['Latitude'], c['Longitude'])
m = folium.Map(location=loc, zoom_start=18)
m.simple_marker(location=loc)
m

In [ ]:
# Update stops with the latlng inferred from annotation
def update(stop):
    if stop['BusStopCode'] not in heading_annotations_by_stop:
        return stop
    
    a = heading_annotations_by_stop[stop['BusStopCode']]
    
    # clone the dict
    d = dict(stop.items())
     
    # update the lat, lng
    (lat1,lng1), (lat2,lng2) = a['heading']
    (lat,lng) = a['Latitude'], a['Longitude']

    xy1, xy2 = svy(lng1, lat1), svy(lng2, lat2)
    x,y = svy(lng, lat)

    # project x,y onto the line xy1-xy2
    length = math.sqrt((xy2[0] - xy1[0]) * (xy2[0] - xy1[0]) + (xy2[1] - xy1[1]) * (xy2[1] - xy1[1]))
    n = (xy2[0] - xy1[0]) / length, (xy2[1] - xy1[1]) / length

    diff = x-xy1[0], y-xy1[1]

    projected_length = np.dot(diff, n)

    px = xy1[0] + n[0] * projected_length
    py = xy1[1] + n[1] * projected_length
    
    upd_lng, upd_lat = svy(px, py, inverse=True)
    
    d['Latitude'] = upd_lat
    d['Longitude'] = upd_lng
    
    print d
    
    return d
     
projected_bus_stops = [update(stop) for stop in BusStops]


In [ ]:
# Update stops with the heading...
def update_heading(stop, headings):
    def zero_threesixty(n):
        while n < 0:
            n += 360
        while n > 360:
            n -= 360
        return n
    
    
    if stop['BusStopCode'] not in headings:
        print stop
        return stop
    
    d = dict(stop.items())
    d['Heading'] = zero_threesixty(headings[stop['BusStopCode']] / math.pi * 180)
    
    return d
    
    
stops_with_heading = [update_heading(stop, all_headings) for stop in projected_bus_stops if stop['Longitude'] != 0]

In [ ]:
[s for s in stops_with_heading if 'Heading' in s][0]

In [ ]:
json.dump(stops_with_heading, open('./bus-stops-headings.json', 'w'))

In [ ]:
def comp_heading(x,y):
    return math.atan2(x, y) / math.pi * 180

comp_heading(0, 1)

In [25]:
points=[(1.29684825487647,103.85253591654006),(1.29770970610083,103.8532247463225),(1.29698951191332,103.85302201172507),(1.2966729849642,103.85441422464267),(1.29807216909408,103.84975607305783),(1.29948166696821,103.85092666701239),(1.2940020309036,103.85128983925452),(1.29276132399953,103.84983741344308),(1.29489484954945,103.85108138663934),(1.29250006522444,103.85145315468638),(1.29208221483052,103.85125994680256),(1.29043611099357,103.84712444398055),(1.29084476137495,103.84857901022669),(1.29044830830726,103.84875123545265),(1.28882717929599,103.8474938118651),(1.29043039362749,103.85026216508201),(1.2897807341128036,103.848908985756),(1.29197057108024,103.84328384973516),(1.29183939431601,103.84504089382705),(1.28118388885484,103.84105000000471),(1.28724656561704,103.84833343750249),(1.28293819129834,103.84254276754574),(1.28531114761754,103.84455070696784),(1.28676420118491,103.84575036447065),(1.28027629959632,103.84029791443083),(1.28398513155185,103.84527387289731),(1.28447078699184,103.84667079234325),(1.28165999996967,103.83801388886192),(1.28607055552165,103.83879166662209),(1.28475980959972,103.83405671725487),(1.28685859481289,103.83360028266564),(1.28837086415387,103.83349881780524),(1.28963300933974,103.83381241921823),(1.28984045728104,103.83411526676929),(1.28870976841461,103.83737276969174),(1.28901705678419,103.83994286096711),(1.2860773006121,103.84244788580838),(1.29734200002733,103.84904099999753),(1.29945446471936,103.85267983857253),(1.3006979578813294,103.85117982031835),(1.29831153783592,103.8452569595532),(1.29811014243867,103.84780636909545),(1.29821382250893,103.84936128375956),(1.29986928715834,103.84093746088925),(1.30027569326586,103.83877618459663),(1.30044251755834,103.83623804216019),(1.28045309423663,103.82638920754815),(1.28583847140468,103.83238179897589),(1.28572048365401,103.8285292287095),(1.28593135445772,103.82668969231534),(1.28599961406266,103.82748123930962),(1.28736808047078,103.82346004890803),(1.28848771148124,103.8209448467392),(1.28860026020096,103.81903121724014),(1.28928144532567,103.81662854691157),(1.29075499997303,103.81144166700754),(1.29032906602668,103.83111793441608),(1.2904161299801,103.82791431393503),(1.2906166672167,103.82362326803131),(1.29090335842692,103.82047232884216),(1.29115999997607,103.81757666701353),(1.29153833298318,103.81445166697797),(1.2918699745378,103.81440639448375),(1.29215855273858,103.81065866068086),(1.29677663321893,103.81007237550017),(1.29652831402079,103.81043015393529),(1.28365072319387,103.824502653383),(1.28906972186602,103.82559411728036),(1.28875832991208,103.82573455047122),(1.29680597432594,103.81169140516062),(1.29666888890258,103.81468924528275),(1.28105845355429,103.82528065411316),(1.29172303079906,103.82610949356565),(1.29328273601932,103.82663492676832),(1.29488000002792,103.82880805602646),(1.29616972198544,103.83255861099337),(1.29575111096804,103.83604111103381),(1.29578024244395,103.83961272081199),(1.29414162678811,103.84231209757696),(1.29350944398488,103.84272416702719),(1.29796011528522,103.83053183555204)]

In [26]:
import folium
m=folium.Map(location=(1.38,103.8), zoom_start=12)

def swap(x,y):
    return (y,x)

for p in points:
    folium.Marker(location=p, popup='{},{}'.format(*p)).add_to(m)
# m.simple_marker((1.28601166699569,103.84634666701143))

start = swap(103.85253591654006,1.29684825487647)
end = swap(103.81468924528275,1.29666888890258)

folium.Marker(icon=folium.Icon(color='red'), location=start).add_to(m)
folium.Marker(icon=folium.Icon(color='red'), location=end).add_to(m)

m.line([[1.2968033896679187, 103.8525876965029],[1.2980506114225971, 103.85359161764961],[1.298528938707611, 103.85397923411382],[1.298453315312862, 103.85407292516693],[1.296832627732291, 103.85279570935955],[1.295942842124615, 103.85197763559172],[1.2948358720876845, 103.85092803502667],[1.2945691412963527, 103.85071532094588],[1.294464460636626, 103.8506316881768],[1.2931762552226242, 103.84992909841081],[1.2925723856517461, 103.84967279843255],[1.2920974111280756, 103.84950292519115],[1.2910049697236337, 103.84897058119559],[1.2904741158442374, 103.84865765680352],[1.2900993516318355, 103.84837099570865],[1.2899896418300936, 103.84828289259113],[1.2894902666544228, 103.84789024698489],[1.2885673259623496, 103.8471662367992],[1.2873804484467308, 103.84614662482173],[1.2872891788323786, 103.84608236356263],[1.2873955358727769, 103.84592813654083],[1.2879895334241436, 103.84502084206837],[1.2888370369859867, 103.84396788874197],[1.289324491232436, 103.84323717303359],[1.2898540412601986, 103.84242654984652],[1.2899415855841694, 103.84221830611419],[1.2899255668355278, 103.84214696680338],[1.2899050777384284, 103.84211753700936],[1.2898465906794352, 103.84206482415046],[1.2885645319945633, 103.8413508722488],[1.2882489998992308, 103.84121452662083],[1.2883281623198426, 103.84111878665801],[1.2886613895444883, 103.8407741972977],[1.2888824855286438, 103.8404832521189],[1.2890277718535312, 103.84022620708257],[1.289091474319059, 103.8399972879886],[1.2890860726480053, 103.8397484385911],[1.2890441631312108, 103.83952752887147],[1.2888232534115744, 103.83893371758462],[1.2887789224560318, 103.8387686872207],[1.2887358953521229, 103.83847904589354],[1.288733846442413, 103.83812532957178],[1.2888415073344448, 103.83688052379073],[1.288932031890721, 103.83631763241405],[1.2890581329701423, 103.83577318122477],[1.2892831405091987, 103.83502383906448],[1.2895910357592486, 103.83415081726352],[1.289914763493421, 103.83332473412136],[1.2900352766372698, 103.83327611908189],[1.2897609339781309, 103.83408839472453]]
)
m

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: FutureWarning: line is deprecated. Use add_children(PolyLine) instead


In [4]:
folium.__version__

'0.1.6'

In [1]:
import folium
folium.__version__

'0.2.1'